# Factoring with Dann5 d5o and D-Wave Ocean and other basic features

This Jupyter Notebook demonstrates:
1.	a programmatic conversion of a problem statement into a Qubo binary quadratic model (BQM) for D-Wave quantum annealing computer (QAC) execution by using [Dann5 d5o library](https://github.com/voya-voja/dann5), 
2.	submission of the Qubo to the D-Wave system and retrieval of corresponding solution samples,  and 
3.	conversion of the retrieved solution samples  into human readable form by using Dann5 d5o library.

In the [Leap factoring demo](https://cloud.dwavesys.com/learning/user/nebojsa_2evojinovic_40rogers_2ecom/notebooks/leap/demos/factoring/01-factoring-overview.ipynb), you saw how the D-Wave system can be used to factor an integer by running a multiplication circuit in reverse. The factoring demo shows how you can solve a [constraint satisfaction problem (CSP)]( https://docs.ocean.dwavesys.com/en/stable/concepts/csp.html) on a QAC. In a way **Dann5 d5o programming framework** is an extension of python programming langue that provides a human friendly generalization of CSP implementation, which permits QAC programmers to use constructs such as data types definitions, equation/condition statements, programmable routines and functions, etc.

So, if you have [installed Dann5 d5o library](https://github.com/voya-voja/dann5/blob/master/README.md) and all prerequisites, lets start.

## Defining d5o quantum variables and equations

In [1]:
from dann5.d5o import Qvar, Qequation

> The line above allows definition of **Q** (quantum) **variables and equations** in the rest of the python code

To define a Q variable **a** as a whole number with **4 Q bits in S**(uperposition) state, it is as simple as:

In [2]:
var = Qvar(4, "a")

> Note that **'var'** is a python variable with a reverence to the defined Q variable **'a'**.

There is no reason that python and Q variable can't have the same name, like in the following line. **A python variable has a reference to a defined Q variable with the same name 'b'**.

In [3]:
b = Qvar(3, "b")

> The Q variable 'b' is of **Qwhole type with U**(ndefined) **value**, meaning that **at least one of its Qbits is in S state**. In case of this definition of Q variable ‘b’, all 3 Qbits are in S state.

Both Q variables, 'a' and 'b' are unknown. If we want to define a deterministic Q variable we specify its initialization value:

In [4]:
A = Qvar("A", 15)

> Like in python, a Q variable 'A' is different from Q variable 'a', i.e. d5o is case sensitive.

The Q variable 'A' is a 4 Qbit Qwhole number with all Qbits set to 1, i.e. binary 1111.

We can now define Q addition equation by initializing Qequation object with the result Q variable:

In [5]:
eA = Qequation(A)

... and assign the addition expression **'a' + 'b'** as a python expression **'var' + 'b'**, as python 'var' variable references a definition of Q variable 'a':

In [6]:
eA.assign(var + b)

In [7]:
print(eA.toString(True, -1))

| A0 = 1 | = a0 ^ b0
| A1 = 1 | = a1 + b1 + #|A0|
| A2 = 1 | = a2 + b2 + #|A1|
| A3 = 1 | = a3 ^ #|A2|
| A4 = 0 | = #|A3|



> The resulting 'eA' Q equation represents a CSP for **'A' = 'a' + 'b'** statement, where 'a' and 'b' are unknown and expected result 'A' is 15.

Now we can convert the Q equation 'eA' to Qubo binary quadratic model (DQM) form 'gQ_eA'. Here we are requesting a **'generic' [Qubo BQM](https://docs.dwavesys.com/docs/latest/c_gs_3.html#qubo)** of 'eA' Q equation, which means that deterministic variables are nor substituted with their values.  

In [8]:
gQ_eA = eA.qubo(False)
print(gQ_eA)

{('#|A0|', '#|A0|'): 5.0, ('#|A0|', '#|A1|'): -4.0, ('#|A0|', 'A1'): -2.0, ('#|A1|', '#|A1|'): 5.0, ('#|A1|', '#|A2|'): -4.0, ('#|A1|', 'A2'): -2.0, ('#|A2|', '#|A2|'): 5.0, ('#|A2|', 'A3'): -2.0, ('#|A2|', 'A4'): -4.0, ('A0', '#|A0|'): 4.0, ('A0', 'A0'): 1.0, ('A1', '#|A1|'): 4.0, ('A1', 'A1'): 1.0, ('A2', '#|A2|'): 4.0, ('A2', 'A2'): 1.0, ('A3', 'A3'): 1.0, ('A3', 'A4'): 4.0, ('A4', 'A4'): 4.0, ('a0', '#|A0|'): -4.0, ('a0', 'A0'): -2.0, ('a0', 'a0'): 1.0, ('a0', 'b0'): 2.0, ('a1', '#|A0|'): 2.0, ('a1', '#|A1|'): -4.0, ('a1', 'A1'): -2.0, ('a1', 'a1'): 1.0, ('a1', 'b1'): 2.0, ('a2', '#|A1|'): 2.0, ('a2', '#|A2|'): -4.0, ('a2', 'A2'): -2.0, ('a2', 'a2'): 1.0, ('a2', 'b2'): 2.0, ('a3', '#|A2|'): 2.0, ('a3', 'A3'): -2.0, ('a3', 'A4'): -4.0, ('a3', 'a3'): 1.0, ('b0', '#|A0|'): -4.0, ('b0', 'A0'): -2.0, ('b0', 'b0'): 1.0, ('b1', '#|A0|'): 2.0, ('b1', '#|A1|'): -4.0, ('b1', 'A1'): -2.0, ('b1', 'b1'): 1.0, ('b2', '#|A1|'): 2.0, ('b2', '#|A2|'): -4.0, ('b2', 'A2'): -2.0, ('b2', 'b2'): 1.0}


To prepare a Qubo BQM of the Q equation 'eA' for processing on QAC, we need to create a **'finalized'** version.

In [9]:
fQ_eA = eA.qubo()
print(fQ_eA)

{('#|A0|', '#|A0|'): 7.0, ('#|A0|', '#|A1|'): -4.0, ('#|A1|', '#|A1|'): 7.0, ('#|A1|', '#|A2|'): -4.0, ('#|A2|', '#|A2|'): 7.0, ('a0', '#|A0|'): -4.0, ('a0', 'a0'): -1.0, ('a0', 'b0'): 2.0, ('a1', '#|A0|'): 2.0, ('a1', '#|A1|'): -4.0, ('a1', 'a1'): -1.0, ('a1', 'b1'): 2.0, ('a2', '#|A1|'): 2.0, ('a2', '#|A2|'): -4.0, ('a2', 'a2'): -1.0, ('a2', 'b2'): 2.0, ('a3', '#|A2|'): 2.0, ('a3', 'a3'): -1.0, ('b0', '#|A0|'): -4.0, ('b0', 'b0'): -1.0, ('b1', '#|A0|'): 2.0, ('b1', '#|A1|'): -4.0, ('b1', 'b1'): -1.0, ('b2', '#|A1|'): 2.0, ('b2', '#|A2|'): -4.0, ('b2', 'b2'): -1.0}


> If we compare the two Qubo BQM's, we will see that the second **does not have any [linear or quadratic pairs](https://docs.dwavesys.com/docs/latest/c_gs_3.html#objective-functions) containing Qbit definitions of deterministic Q variable(s)**, i.e. Q variable 'A' Qbit's definitions {A0 - A4} have been substituted with their values in 'generic' Qubo BQM and the resulting Qubo BQM has been normalized to make a 'finalized' version of Qubo BQM for Q equation 'eA'.

## Solving Q equation and reporting results

To solve our problem statement *'A' = 'a' + 'b', where 'A' = 15 and 'a'='b'=U(nknown)* defined by Q equation *'eA'* we will use [D-Wave's Exact Solver](https://docs.ocean.dwavesys.com/projects/dimod/en/0.7.0/reference/generated/dimod.reference.samplers.ExactSolver.sample.html):

In [10]:
from dimod import ExactSolver
exactSolver = ExactSolver()                   # local solver

... and execute request *sample_qubo()* to solve *'fQ_eA'*, a 'finalized' Qubo BQM for 'eA' Q equation.

In [11]:
sampleset = exactSolver.sample_qubo(fQ_eA)

The **D-Wave sampleset has to be converted into a python dictionary** ('samples'), before it is passed to the Q equation 'eA' to provide solutions in a human readable form: 

In [12]:
samples = [dict(sample) for sample in sampleset.lowest().samples()]
eA.set(samples)
print(eA.solutions())

a = 10; b = 5; 
a = 11; b = 4; 
a = 8; b = 7; 
a = 9; b = 6; 
a = 14; b = 1; 
a = 15; b = 0; 
a = 12; b = 3; 
a = 13; b = 2; 



> Note that we have used **sampleset.lowest().samples()** to ensure that from teh whole sampleset of all posible solution of Qubo BQM, we retrieve only those with the lowest energy level as the best solution for our CSP described by eA Q equation.

By carefully reviewing the returned results you will see that *8 <= 'a' <= 15* and *0 <= 'b' <= 7*, which are appropriate results considering that we set Q variable 'b' to have only 3 Qbits, i.e. imposing additional condition that 'b' cannot be bigger than 7.

> If you go back and change only the number of Qbits for variable 'b', the solution set for Q equation eA will be different

## Factoring with d5o

Back to the [Leap factoring demo](https://cloud.dwavesys.com/learning/user/nebojsa_2evojinovic_40rogers_2ecom/notebooks/leap/demos/factoring/01-factoring-overview.ipynb), the same problem can be represented in d5o as *P = x * y; where P = 21 and 'x' and 'y' are 3 Qbits unknown Q whole numbers*:

In [13]:
x = Qvar(3, "x")
y = Qvar(3, 'y')
eP = Qequation(Qvar("P", 21))
eP.assign(x * y)
print(eP.toString())

| P0 = 1 | = x0 & y0
| P1 = 0 | = x0 & y1 ^ x1 & y0
| P2 = 1 | = x0 & y2 + #|P1| + x1 & y1 ^ x2 & y0
| P3 = 0 | = x1 & y2 + #|x0 & y2 + #|P1| + x1 & y1| + #|P2| ^ x2 & y1
| P4 = 1 | = x2 & y2 + #|x1 & y2 + #|x0 & y2 + #|P1| + x1 & y1| + #|P2|| + #|P3|
| P5 = 0 | = #|P4|



The next line prints 'eP' Q equation decomposed in elementary circuit logic:

In [14]:
print(eP.toString(True))

| P0 = 1 | = x0 & y0
| P1 = 0 | = &|100| ^ &|110|
	&|100| = x0 & y1
	&|110| = x1 & y0
| P2 = 1 | = +|200| ^ &|210|
	+|200| = &|2001| + #|P1| + &|2021|
	&|2001| = x0 & y2
	&|2021| = x1 & y1
	&|210| = x2 & y0
| P3 = 0 | = +|300| ^ &|310|
	+|300| = &|3001| + #|+|200|| + #|P2|
	&|3001| = x1 & y2
	&|310| = x2 & y1
| P4 = 1 | = &|400| + #|+|300|| + #|P3|
	&|400| = x2 & y2
| P5 = 0 | = #|P4|



> The elementary circuits and Qbit definitions of Q variables are the nodes of Qubo BQM for 'eP' Q equation.

In [15]:
print(eP.qubo(False))

{('#|P1|', '#|P1|'): 5.0, ('#|P1|', '#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|'): -4.0, ('#|P1|', 'x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1'): -2.0, ('#|P1|', 'x1 & y1'): 2.0, ('#|P2|', '#|P2|'): 5.0, ('#|P2|', '#|x1 & y2 + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1| + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1 ^ x2 & y0||'): -4.0, ('#|P2|', 'x1 & y2 + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1| + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1 ^ x2 & y0|'): -2.0, ('#|P3|', '#|P3|'): 5.0, ('#|P3|', 'P4'): -2.0, ('#|P3|', 'P5'): -4.0, ('#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|', '#|P2|'): 2.0, ('#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|', '#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|'): 5.0, ('#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|', '#|x1 & y2 + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1| + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1 ^ x2 & y0||'): -4.0, ('#|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1|', 'x1 & y2 + #|x0 & y2 + #|x0 & y1 ^ x1 & y0| + x1 & y1| + #|x0 & y2 + #|x0 & y1 ^

In [16]:
fQ_eP = eP.qubo()

In [17]:
sampleset = exactSolver.sample_qubo(fQ_eP)

In [18]:
samples = [dict(sample) for sample in sampleset.lowest().samples()]
eP.set(samples)
print(eP.solutions())

x = 3; y = 7; 
x = 7; y = 3; 



## Subtraction and Division

d5o allows QAC developers to define a subtraction Q equations... 

In [19]:
a = Qvar(4, 'a')
T = Qvar('T', 7)
eS = Qequation(T - a)
sampleset = exactSolver.sample_qubo(eS.qubo(True, -1))
samples = [dict(sample) for sample in sampleset.lowest().samples()]
eS.set(samples)
print(eS.solutions())

dfrnc = 0; a = 7; 
dfrnc = 1; a = 6; 
dfrnc = 3; a = 4; 
dfrnc = 2; a = 5; 
dfrnc = 6; a = 1; 
dfrnc = 7; a = 0; 
dfrnc = 5; a = 2; 
dfrnc = 4; a = 3; 



... or a division equation.

In [20]:
d = Qvar(3, 'd')
M = Qvar("M", 15)
eD = Qequation(M / d)
sampleset = exactSolver.sample_qubo(eD.qubo())
samples = [dict(sample) for sample in sampleset.lowest().samples()]
eD.set(samples)
print(eD.solutions())

qtnt = 3; d = 5; 
qtnt = 5; d = 3; 



## Using Constants in a Q equation
A definition and use of a constant in d5o is same as definition of any other variable with a deterministic value. In order to easily distinguish constants from variables we recommend following syntax as an example of best coding practice:

> _3 = Qvar("3_", 3)

Based on above we can make a Q equation such as:

In [30]:
_3 = Qvar("3_", 3)
i = Qvar(2, "i")
j = Qvar(2, "j")
C = Qvar("C", 15)
eC = Qequation(C)
eC.assign(_3 * (i + j))
sampleset = exactSolver.sample_qubo(eC.qubo())
samples = [dict(sample) for sample in sampleset.lowest().samples()]
eC.set(samples)
print(eC.solutions())

i = 2; j = 3; 3_ = 3; 
i = 3; j = 2; 3_ = 3; 

